In [11]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img , img_to_array
import os
import uuid
import shutil
import glob
from sklearn.metrics import classification_report, accuracy_score

In [3]:
tf.keras.backend.clear_session()

## Data Augmentation

In [376]:

# #copy some images of cars to the test folder
# def copy_files(source_path, destination_path, num_files=5):
#     # Iterate over each directory in the source path
#     for root, dirs, files in os.walk(source_path):
#         #files = files[:10]
#         # Iterate over each file in the current directory
#         for i, file_name in enumerate(files):
#             # Check if we've copied the desired number of files already
#             if i >= num_files:
#                 break
            
#             # Generate a unique UUID for the new file name
#             unique_id = str(uuid.uuid4())
            
#             # Construct the source and destination file paths
#             source_file_path = os.path.join(root, file_name)
#             destination_file_name = unique_id + os.path.splitext(file_name)[1]  # Preserve file extension
#             destination_file_path = os.path.join(destination_path, destination_file_name)
            
#             # Copy the file to the destination
#             shutil.copy(source_file_path, destination_file_path)

# source_path = "E:/Data/Cars/car_data/car_data/train"
# destination_path = "D:/ML Projects/PPP Model/CategoryModel/Train/Cars"
# copy_files(source_path, destination_path, num_files=12)


In [377]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import img_to_array

# # Define data augmentation parameters (adjust as needed)
# datagen = ImageDataGenerator(
#     rotation_range=60,  # Rotate images by up to 40 degrees
#     shear_range=0.4,     # Shear by up to 20%
#     zoom_range=0.3,      # Zoom by up to 20%
#     horizontal_flip=True, # Randomly flip images horizontally
#     brightness_range=(0.5, 1.5)  # Adjust brightness range
# )

# # Define path to your image folder
# image_dir = "E:/Data/watches/images.cv_3v5apop148okjqfbzufach/data/train/wristwatch"

# # Define output directory for augmented images (create if it doesn't exist)
# save_to_dir = "Train/Clothes"
# import os
# os.makedirs(save_to_dir, exist_ok=True)  # Create directory if it doesn't exist

# # Loop through each image in the folder
# for filename in os.listdir(image_dir):
#     # Load the image
#     img = load_img(os.path.join(image_dir, filename))

#     # Convert to array
#     x = img_to_array(img)

#     # Reshape (optional, based on your model's requirements)
#     x = x.reshape((1,) + x.shape)  # Reshape if your model expects a specific format

#     # Generate and save augmented images
#     i = 0
#     for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix="aug_", save_format="jpeg"):
#         i += 1
#         # You can adjust the number of augmented images per original image here (e.g., i > 5)
#         if i > 2:
#             break
# print(f"Successfully augmented and saved images to {save_to_dir}")


In [378]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [379]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(6,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [380]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [381]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# ... your model definition ...

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(from_logits=True),  # Adjust if needed
              metrics=['accuracy'])

In [6]:
train_Path = "Train"
IMAGE_SIZE = (224,224)
folders = glob.glob(train_Path + '/*')
folders

['Train\\Animals',
 'Train\\Cars',
 'Train\\Clothes',
 'Train\\Furniture',
 'Train\\It',
 'Train\\Jewellery']

In [7]:
labels = ["Animals","Cars","Clothes","Furniture","It","Jewellery"]

In [385]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
training_set = train_datagen.flow_from_directory(
    train_Path,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical', # As we have more than 2 so using categorical.. for 2 we might have used binary.
    shuffle = True
)


Found 18692 images belonging to 6 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_Path = "Test"
test_set = test_datagen.flow_from_directory(
    test_Path,  # Use your test data directory here
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle= False
)

Found 1751 images belonging to 6 classes.


In [387]:
model.fit(training_set, epochs=10, steps_per_epoch=len(training_set),validation_data=test_set)

Epoch 1/10
397/585 ━━━━━━━━━━━━━━━━━━━━ 2:41 857ms/step - accuracy: 0.8278 - loss: 0.4838

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


585/585 ━━━━━━━━━━━━━━━━━━━━ 521s 874ms/step - accuracy: 0.8597 - loss: 0.3994 - val_accuracy: 0.9760 - val_loss: 0.0761
Epoch 2/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9760 - val_loss: 0.0769
Epoch 3/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 391s 664ms/step - accuracy: 0.9707 - loss: 0.0902 - val_accuracy: 0.9806 - val_loss: 0.0628
Epoch 4/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9806 - val_loss: 0.0625
Epoch 5/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 404s 687ms/step - accuracy: 0.9778 - loss: 0.0676 - val_accuracy: 0.9794 - val_loss: 0.0671
Epoch 6/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9794 - val_loss: 0.0700
Epoch 7/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 392s 665ms/step - accuracy: 0.9845 - loss: 0.0505 - val_accuracy: 0.9829 - val_loss: 0.0644
Epoch 8/10
585/585 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.0

In [181]:
pred_test = model.predict(test_set, steps=len(test_set), verbose=0)

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
test_labels = test_set  # This is a generator
all_test_labels = []
for _, labels in test_set:
  all_test_labels.extend(labels.tolist())  # Convert labels in each batch to list


In [19]:
# Make predictions
predictions = model.predict(x)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])  # Assuming one prediction per image
max = np.max(predictions[0])
if(max<0.7):
    print("Other")
else:
    print(labels[predicted_class])
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Other
[[0.20119467 0.02955429 0.08404132 0.00348682 0.67595965 0.00576322]]


In [12]:
# model.save("CategoryModelV2.keras")

In [9]:
model = tf.keras.models.load_model("CategoryModelV2.keras")

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 326 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
test_labels = test_set.classes

# Predict the labels for the test set
predictions = model.predict(test_set, steps=len(test_set))
predicted_classes = np.argmax(predictions, axis=1)

# Get the class indices to labels mapping
class_labels = list(test_set.class_indices.keys())

# Print classification report
report = classification_report(test_labels, predicted_classes, target_names=class_labels)
print(report)

55/55 ━━━━━━━━━━━━━━━━━━━━ 24s 427ms/step
              precision    recall  f1-score   support

     Animals       0.98      1.00      0.99       632
        Cars       0.99      1.00      1.00       392
     Clothes       0.98      0.90      0.94       207
   Furniture       0.78      1.00      0.88        65
          IT       0.97      0.93      0.95       360
   Jewellery       1.00      1.00      1.00        95

    accuracy                           0.97      1751
   macro avg       0.95      0.97      0.96      1751
weighted avg       0.98      0.97      0.97      1751

